<h1> Design of parametric multistorey house </h1>
<h5>Nicola Sardella - 440648 - workshop No. 10</h5>

In [7]:
from tools import *
from stair_landings import *
from windows_and_doors import *
from house_structure import *
from roof import *



<h2>Description:</h2>
In questo workshop utilizzerò i workshop precedenti per realizzare una casa completa di porte, finestre, scale e tetto. <br>In particolare i workshop utilizzati sono:
* workshop_03 per creare le scale [stair_landing.py]
* workshop_07 per realizzare le porte e finestre [windows_and_doors.py]
* workshop_08 per realizzare la struttura della casa (muri esterni, muri interni e pavimento) [house_structure.py]
* workshop_09 per realizzare il tetto [roof.py]

<br>Come si nota da i file importati, ogni workshop è stato riscritto e copiato in un file .py, per poi essere eseguito in questo workshop.

<br>Inoltre viene utilizzato un altro plugin (tools.py) realizzato da me, contenente diverse funzioni di supporto per questo e i precedenti workshop

<h2>Geometric method:</h2>
<br>Lo scopo è quello di realizzare una casa completa a due piani, partendo da due planimetrie (quella del primo piano e quella del secondo piano).
![primo](immagini/primo_piano.jpg)
![secondo](immagini/secondo_piano.jpg)
<br>Partendo da queste due planimetrie vengono generati vari file .lines.<br>In particolare:
* Un file .lines per i muri esterni
* Un file .lines per i muri interni
* Un file .lines per le finestre
* Un file .lines per le porte 
* Un file .lines per le finestre
* Un file .lines per la posizione delle scale
* Vari file .lines per le varie zone del pavimento che hanno la stessa texture



Una volta ottenuti questi file .lines il programma li elabora, in diversi passaggi, generando alla fine un output contenente un oggetto HPC che descrive una casa completa di porte, finestre, scale e tetto. 

<h2>Algorithm</h2>
Per realizzare la funzione finale vanno eseguiti diversi passaggi, utilizzando funzioni di supporto. <br>In particolare:
1. Per prima cosa bisogna determinare i "dati" di porte, finestre e scale.
    * Per **dati** si intendono le dimensioni e la posizione di ciascun elemento
    * Le funzioni di supporto per fare questa operazione sono quelle denominate **getData...**
2. Una volta ottenuti i ***dati*** di porte, finestre e scale, questi andranno usati per creare un l'oggetto HPC corrispondente (per ogni porta, finestra e scala).
    * Le funzioni di supporto per fare questa operazione sono quelle denominate **adapt...**
3. Il passaggio successivo è quello di **creare il tetto**
    * Questa operazione viene fatta utilizzando la funzione ***roofCreator*** che prende in input il primo e il secondo piano
4. In fine possiamo creare la funzione finale ***desingHouse*** per costruire la casa finale, utilizzando tutte le funzioni di supporto precedentemente create

<h2> Funzioni di Supporto </h2>
<h3> 1. Estrarre dimensioni e posizione con getData </h3>

In [8]:
def getDataExternalDoors (dimX,dimY,externalLinesPath,doorsLinePath):
	"""
	getDataExternalDoors is a function that generate an Array, containing, for each doors, 
	its dimension and position on 3d plane xyz.
	@input dimX: The dimensions along the x axis of the external wall
	@input dimY: The dimensions along the y axis of the external wall
	@input externalLinesPath: path of .lines file of the external wall.
	@input doorslLinesPath: path of .lines file of the doors.
	@return [[dimensionDoor1,positionDoor1],[dimensionDoor2,positionDoor2],...]: 
	An array containing dimension and position for each door 
	"""
	#genero i muri esterni
	externalWalls = building_wall(externalLinesPath,6,3)
	#calcolo i fattori di scala x e y 
	xfactor=dimX/SIZE([1])(externalWalls)[0]
	yfactor=dimY/SIZE([2])(externalWalls)[0]
	#ridimensiono
	externalWalls = (S([1,2])([xfactor,yfactor])(externalWalls))
	#creo cubi dove andranno le porte e li metto, uno ad uno, in un array
	with open(doorsLinePath, "rb") as file:
		reader = csv.reader(file, delimiter=",")
		doorsList = []
		cuboid = []
		acc = 0
		for row in reader:
			acc = acc + 1
			cuboid.append([float(row[0]),float(row[1])])
			if(acc == 4):
				cub=STRUCT([MKPOL([cuboid,[[1,2,3,4]],None])])
				cub = PROD([cub, Q(2.5)])
				cub = (S([1,2])([xfactor,yfactor])(cub))
				doorsList.append(cub)
				cuboid = []
				acc = 0

	#per ogni cubo mi trovo la corrispettiva porta e calcolo la dimensione e la posizione 
	dataDim=[]

	for c in doorsList:
		externalWalls2=DIFFERENCE([externalWalls,c])
		door = DIFFERENCE([externalWalls,externalWalls2])
		sizeDoor = SIZE([1,2])(door)
		if (sizeDoor[0] != 0.0 and sizeDoor[1] != 0.0):
			#calcolo la posizione e la dimensione delle porte esterne
			dataDim.append(getDimensionAndPosition(door))

	return dataDim
	
def getDatainternalDoors(dimX,dimY,externalLinesPath,internalLinesPath,doorsLinePath):
	"""
	getDataInternalDoors is a function that generate an Array, containing, for each doors, 
	its dimension and position on 3d plane xyz.
	@input dimX: The dimensions along the x axis of the external wall
	@input dimY: The dimensions along the y axis of the external wall
	@input externalLinesPath: path of .lines file of the external wall.
	@input internalLinesPath: path of .lines file of the internal wall.
	@input doorslLinesPath: path of .lines file of the doors.
	@return [[dimensionDoor1,positionDoor1],[dimensionDoor2,positionDoor2],...]: 
	An array containing dimension and position for each door 
	"""
	#genero i muri interni
	externalWalls = building_wall(externalLinesPath,6,3)
	internalWalls = building_wall(internalLinesPath,3,3)
	internalWalls = STRUCT([externalWalls,internalWalls])
	#calcolo i fattori di scala x e y 
	xfactor=dimX/SIZE([1])(internalWalls)[0]
	yfactor=dimY/SIZE([2])(internalWalls)[0]
	#ridimensiono
	internalWalls = (S([1,2])([xfactor,yfactor])(internalWalls))
	externalWalls = (S([1,2])([xfactor,yfactor])(externalWalls))
	internalWalls=DIFFERENCE([internalWalls,externalWalls])
	#creo cubi dove andranno le porte e li metto, uno ad uno, in un array
	with open(doorsLinePath, "rb") as file:
		reader = csv.reader(file, delimiter=",")
		doorsList = []
		cuboid = []
		acc = 0
		for row in reader:
			acc = acc + 1
			cuboid.append([float(row[0]),float(row[1])])
			if(acc == 4):
				cub=STRUCT([MKPOL([cuboid,[[1,2,3,4]],None])])
				cub = PROD([cub, Q(2.5)])
				cub = (S([1,2])([xfactor,yfactor])(cub))
				doorsList.append(cub)
				cuboid = []
				acc = 0

	#per ogni cubo mi trovo la corrispettiva porta e calcolo la dimensione e la posizione 
	dataDim=[]

	for c in doorsList:
		internalWalls2=DIFFERENCE([internalWalls,c])
		door = DIFFERENCE([internalWalls,internalWalls2])
		sizeDoor = SIZE([1,2])(door)
		if (sizeDoor[0] != 0.0 and sizeDoor[1] != 0.0):
			#calcolo la posizione e la dimensione delle porte interne
			dataDim.append(getDimensionAndPosition(door))

	return dataDim

from house_structure import *

def getDataWindows (dimX,dimY,externalLinesPath,windowsLinePath):
	"""
	getDataWindowa is a function that generate an Array, containing, for each window, 
	its dimension and position on 3d plane xyz.
	@input dimX: The dimensions along the x axis of the external wall
	@input dimY: The dimensions along the y axis of the external wall
	@input externalLinesPath: path of .lines file of the external wall.
	@input windowsLinesPath: path of .lines file of the windows.
	@return [[dimensionWindow1,positionWindow1],[dimensionWindow2,positionWindow2],...]: 
	An array containing dimension and position for each window 
	"""
	#genero i muri esterni
	externalWalls = building_wall(externalLinesPath,6,3)
	#calcolo i fattori di scala x e y 
	xfactor=dimX/SIZE([1])(externalWalls)[0]
	yfactor=dimY/SIZE([2])(externalWalls)[0]
	#ridimensiono
	externalWalls = (S([1,2])([xfactor,yfactor])(externalWalls))
	#creo cubi dove andranno le finestre e li metto, uno ad uno, in un array
	with open(windowsLinePath, "rb") as file:
		reader = csv.reader(file, delimiter=",")
		windowsList = []
		cuboid = []
		acc = 0
		for row in reader:
			acc = acc + 1
			cuboid.append([float(row[0]),float(row[1])])
			if(acc == 4):

				cub=STRUCT([MKPOL([cuboid,[[1,2,3,4]],None])])
				cub = PROD([cub, Q(SIZE([3])(externalWalls)[0]/2.)])
				cub = T(3)(SIZE([3])(externalWalls)[0]/4.)(cub)
				cub = (S([1,2])([xfactor,yfactor])(cub))
				windowsList.append(cub)
				cuboid = []
				acc = 0

	#per ogni cubo mi trovo la corrispettiva finestra e calcolo la dimensione e la posizione 
	dataDim=[]

	for c in windowsList:
		externalWalls2=DIFFERENCE([externalWalls,c])
		window = DIFFERENCE([externalWalls,externalWalls2])
		sizeDoor = SIZE([1,2])(window)
		if (sizeDoor[0] != 0.0 and sizeDoor[1] != 0.0):
			#calcolo la posizione e la dimensione delle finestre
			dataDim.append(getDimensionAndPosition(window))


	return dataDim

def getDataStairs(dimX,dimY,externalLinesPath,internalLinesPath,stairsLinePath):
	"""
	getDataStairs is a function that generate an Array, containing, for each stairs, 
	its dimension and position on 3d plane xyz.
	@input dimX: The dimensions along the x axis of the external wall
	@input dimY: The dimensions along the y axis of the external wall
	@input externalLinesPath: path of .lines file of the external wall.
	@input internallLinesPath: path of .lines file of the internal wall.
	@input stairsLinesPath: path of .lines file of the stairs.
	@return [[dimensionStairs1,positionStairs1],[dimensionStairs2,positionStairs2],...]: 
	An array containing dimension and position for each stairs 
	"""
	#genero tutti i muri della casa
	externalWalls = building_wall(externalLinesPath,6,3)
	internalWalls = building_wall(internalLinesPath,3,3)
	internalWalls = STRUCT([externalWalls,internalWalls])
	#calcolo i fattori di scala x e y 
	xfactor=dimX/SIZE([1])(internalWalls)[0]
	yfactor=dimY/SIZE([2])(internalWalls)[0]
	#ridimensiono
	walls = (S([1,2])([xfactor,yfactor])(internalWalls))
	#creo cubi dove andranno le scale
	with open(stairsLinePath, "rb") as file:
		reader = csv.reader(file, delimiter=",")
		doorsList = []
		cuboid = []
		acc = 0
		for row in reader:
			acc = acc + 1
			cuboid.append([float(row[0]),float(row[1])])
			if(acc == 4):
				cub=STRUCT([MKPOL([cuboid,[[1,2,3,4]],None])])
				cub = PROD([cub, Q(3)])
				cub = (S([1,2])([xfactor,yfactor])(cub))
				doorsList.append(cub)
				cuboid = []
				acc = 0

	#per ogni cubo mi trovo la posizione e dimensione esatta delle scale
	dataDim=[]

	for c in doorsList:
		stair=DIFFERENCE([c,walls])
		sizeStair = SIZE([1,2])(stair)
		if (sizeStair[0] != 0.0 and sizeStair[1] != 0.0):
			#calcolo la posizione e la dimensione delle scale
			dataDim.append(getDimensionAndPosition(stair))

	return dataDim



<h2> Funzioni di Supporto </h2>
<h3> 2. Creazione di finestre, porte e scale con adapt </h3>

In [9]:
def adaptWindow(data):
	"""
	adaptWindow is a function that generate an HTC object that represents all windows for each position 
	and size in the array data.
	@input data: An array containing, for each window, dimension and position.
	@return windows: an HTC object that represents all windows for each position 
	and size in the array data.
	"""
	#data=getDataWindows(dimX,dimY,externalLinesPath,windowsLinesPath)
	windows=[]
	for d in data:
		print(d[0])
		if d[0][0]>d[0][1]:
			w=createWindow(d[0][0],d[0][1],d[0][2])
			w=rotation(w,2)
		else:
			w=createWindow(d[0][1],d[0][0],d[0][2])
			w=rotation(w,1)        
		w=STRUCT([T([1,2,3])(d[1]),w])
		windows.append(w)
	return STRUCT(windows)


def adaptDoor(data):
	"""
	adaptDoor is a function that generate an HTC object that represents all doors for each position 
	and size in the array data.
	@input data: An array containing, for each door, dimension and position.
	@return doors: an HTC object that represents all doors for each position 
	and size in the array data.
	"""
	#data=getDataExternalDoors(dimX,dimY,externalLinesPath,doorsLinesPath)
	doors=[]
	for d in data:
		if d[0][0]>d[0][1]:
			if d[0][0]>=1.5:
				door=createDubleDoor(d[0][0],d[0][1],d[0][2],color(255,255,255))
			else:
				door=createSingleDoor(d[0][0],d[0][1],d[0][2],color(255,255,255))
			door=rotation(door,2)
		else:
			if d[0][1]>=1.5:
				door=createDubleDoor(d[0][0],d[0][1],d[0][2],color(255,255,255))
			else:
				door=createSingleDoor(d[0][0],d[0][1],d[0][2],color(255,255,255))
			door=rotation(door,1)
			
		door=STRUCT([T([1,2,3])(d[1]),door])
		doors.append(door)
	return STRUCT(doors)


def adaptStairs(data):
	"""
	adaptStairs is a function that generate an HTC object that represents all stairs for each position 
	and size in the array data.
	@input data: An array containing, for each stairs, dimension and position.
	@return stairs: an HTC object that represents all stairs for each position 
	and size in the array data.
	"""
	#data=getDataStairs(dimX,dimY,externalLinesPath,internalLinesPath,stairsLinesPath)
	stairs = []
	for d in data:
		if d[0][0]>d[0][1]:
			stair=ggpl_stair_landings(d[0][1],d[0][0],d[0][2])
			stair=rotation(stair,3)
		else:
			stair=ggpl_stair_landings(d[0][0],d[0][1],d[0][2])
			stair=rotation(stair,4)
		stair=STRUCT([T([1,2,3])(d[1]),stair])
		stairs.append(stair)
	return STRUCT(stairs)

<h2> Funzioni di Supporto </h2>
<h3> 3. Creazione del tetto con roofCreator</h3>

In [10]:
def roofCreator(firstFloor, secondFloor):
	dim1=getDimensionAndPosition(firstFloor)
	dim2=getDimensionAndPosition(secondFloor)
	dim=[[dim1[0][0]-dim2[0][0],9,0],[dim2[1][0]+dim2[0][0],dim2[1][1],0]]

	punti=[]
	punti.append([dim[1][0],dim[1][1]])
	punti.append([dim[1][0],dim[1][1]+dim[0][1]])
	punti.append([dim[0][0]+dim[1][0],dim[0][1]+dim[1][1]])
	punti.append([dim[1][0]+dim[0][0],dim[1][1]])
	#print(punti)

	altezza=getMinDistPitch(punti)/2
	tetto=createRoof(punti,PI/5,altezza)

	punti=[]
	punti.append([dim2[1][0],dim2[1][1]])
	punti.append([dim2[1][0],dim2[1][1]+dim2[0][1]])
	punti.append([dim2[0][0]+dim2[1][0],dim2[0][1]+dim2[1][1]])
	punti.append([dim2[1][0]+dim2[0][0],dim2[1][1]])
	#print(punti)

	altezza2=getMinDistPitch(punti)/2
	tetto2=createRoof(punti,PI/5,altezza2)
	
	roof = STRUCT([tetto,T(3)(3),tetto2])
	roof = STRUCT([T(3)(3),roof])
	return roof


<h2> Funzioni finale </h2>
<h3> 4. Creazione della casa completa con desingHouse</h3>

In [11]:
def completeFirstFloor(firstFloorStruct, dataWindows, dataExternalDoor, dataInternalDoor, dataStairs):
	firstFloor = STRUCT([firstFloorStruct,adaptWindow(dataWindows),adaptDoor(dataInternalDoor),adaptDoor(dataExternalDoor),adaptStairs(dataStairs)])
	return firstFloor

def completeSecondFloor(secondFloorStruct,dataWindows,dataInternalDoor):
	
	secondFloor=STRUCT([secondFloorStruct,adaptWindow(dataWindows),adaptDoor(dataInternalDoor)])
	return secondFloor



In [12]:
def desingHouse(externalLinesPath,internalLinesPath,externalLinesPathSecond,internalLinesPathSecond):
	def houseElements(windowsLinesPath,doorsLinesPath,stairsLinesPath,windowsLinesPathSecond,doorsLinesPathSecond):
		def setDim(firstX,firstY,secondX,secondY):
			firstFloorStruct = ggpl_building_house(firstX,firstY,externalLinesPath,internalLinesPath,windowsLinesPath,doorsLinesPath)
			secondFloorStruct = ggpl_building_house_second_floor(secondX,secondY,externalLinesPathSecond,internalLinesPathSecond,windowsLinesPathSecond,doorsLinesPathSecond)

			dataWindows1 = getDataWindows(firstX,firstY,externalLinesPath,windowsLinesPath)
			dataExternalDoor1 = getDataExternalDoors(firstX,firstY,externalLinesPath,doorsLinesPath)
			dataInternalDoor1 = getDatainternalDoors(firstX,firstY,externalLinesPath,internalLinesPath,doorsLinesPath)
			dataStairs1 = getDataStairs(firstX,firstY,externalLinesPath,internalLinesPath,stairsLinesPath)
			first =  completeFirstFloor(firstFloorStruct, dataWindows1, dataExternalDoor1, dataInternalDoor1, dataStairs1)
			first = onAxes(first)
			secondFloorStruct = ggpl_building_house_second_floor(secondX,secondY,externalLinesPathSecond,internalLinesPathSecond,windowsLinesPathSecond,doorsLinesPathSecond)
			dataWindows2= getDataWindows(secondX,secondY,externalLinesPathSecond,windowsLinesPathSecond)
			dataInternalDoor2= getDatainternalDoors(secondX,secondY,externalLinesPathSecond,internalLinesPathSecond,doorsLinesPathSecond)
			second = completeSecondFloor(secondFloorStruct,dataWindows2,dataInternalDoor2)
			second = onAxes(second)
			roof = roofCreator(first,second)
			roof = onAxes(roof)
			column = CUBOID([0.3,0.3,3])
			column = TEXTURE(["texture/wall_internal.jpg", FALSE, TRUE, 1, 1, 0, 2, 2])(column)
			house = STRUCT([first,T(3)(3),second])
			house = STRUCT([house,roof])
			house = rotation(house,2)
			house = STRUCT([house,column])
			return house
		return setDim
	return houseElements

In [13]:
def main():


	externalLinesPath= "lines/esterno.lines"
	internalLinesPath= "lines/interno.lines"
	windowsLinesPath ="lines/finestre.lines"
	doorsLinesPath = "lines/porte.lines"
	stairsLinesPath = "lines/scala.lines"

	externalLinesPathSecond= "lines/esterno2.lines"
	internalLinesPathSecond= "lines/interno2.lines"
	windowsLinesPathSecond="lines/finestre2.lines"
	doorsLinesPathSecond = "lines/porte2.lines"


	house = desingHouse(externalLinesPath,internalLinesPath,externalLinesPathSecond,internalLinesPathSecond)(windowsLinesPath,doorsLinesPath,stairsLinesPath,windowsLinesPathSecond,doorsLinesPathSecond)(19,9,16.3,9)


	VIEW(house)

if __name__ == "__main__":
    main()

[2.133246421813965, 0.1759815216064453, 1.5]
[0.7997732162475586, 0.17451810836791992, 1.5]
[2.10067081451416, 0.17594575881958008, 1.5]
[0.8959465026855469, 0.17462396621704102, 1.5]
[1.0174331665039062, 0.1747570037841797, 1.5]
[0.16990089416503906, 2.1497905254364014, 1.5]
[1.0129356384277344, 0.17345428466796875, 1.5]
[2.111771583557129, 0.17345428466796875, 1.5]
[0.16989898681640625, 0.7028169631958008, 1.5]
[0.16990089416503906, 0.7028160095214844, 1.5]
[2.0854849815368652, 0.17345428466796875, 1.5]
[2.1041879653930664, 0.17352962493896484, 1.5]
[0.7719240188598633, 0.17352962493896484, 1.5]
[0.866908073425293, 0.17352867126464844, 1.5]
[2.0962324142456055, 0.17352962493896484, 1.5]
[0.8502187728881836, 0.17352914810180664, 1.5]
[1.0151329040527344, 0.17352914810180664, 1.5]
[0.7618613243103027, 0.17352867126464844, 1.5]
[2.0575432777404785, 0.17352914810180664, 1.5]


<h2>Results</h2>
![ris1](immagini/primo.PNG)
![ris2](immagini/secondo.PNG)
![ris3](immagini/completa.PNG)